In [1]:
print("okay")

okay


In [2]:
%pwd

'c:\\Users\\PC\\Desktop\\Allproject\\ScheduleAI\\ScheduleAI\\notebooks'

In [3]:
import os
os.chdir("../../")

In [4]:
%pwd

'c:\\Users\\PC\\Desktop\\Allproject\\ScheduleAI'

In [5]:
import json
file_path="./rawdata/new_instruct.json"
with open(file_path, 'r') as f:
    data=json.load(f)
print(data)

{'samples': [{'instruction': 'Explain how an LLM Twin can enhance communication efficiency.', 'answer': "An LLM Twin enhances communication efficiency by automating responses and generating content that aligns with the user's established voice. This capability not only saves time but also ensures that the quality and tone of communication remain consistent. As a result, individuals can focus on more strategic tasks while the LLM Twin manages routine interactions, thus streamlining their overall workflow."}, {'instruction': 'Outline the potential ethical concerns associated with AI technology.', 'answer': 'The potential ethical concerns associated with AI technology include bias in algorithms, job displacement, and the misuse of AI for surveillance. Bias can arise if the training data is not representative, leading to unfair outcomes. Job displacement is a significant concern as automation may replace certain jobs, necessitating a focus on reskilling workers. Additionally, the use of AI

In [6]:
from ScheduleAI.components.embedding import Embedding
obj=Embedding(data["samples"])
output=obj.document_loader()
print(len(output))

c:\Users\PC\Desktop\Allproject\ScheduleAI\ScheduleAI\components\embedding.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.model= HuggingFaceEmbeddings(
c:\Users\PC\Desktop\Allproject\ScheduleAI\schedule\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2500


In [7]:
splitted_doc=obj.splitter()
print(len(splitted_doc))

3186


In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},      # or "cuda" if GPU available
    encode_kwargs={"normalize_embeddings": True}
)
Embedding=model.embed_documents(["Hello world", "Bye bye"])
print(Embedding)

[[-0.03447731211781502, 0.03102317824959755, 0.006734973285347223, 0.026108987629413605, -0.03936202451586723, -0.16030248999595642, 0.06692396104335785, -0.006441520527005196, -0.047450557351112366, 0.014758833684027195, 0.0708753690123558, 0.05552764981985092, 0.019193340092897415, -0.026251304894685745, -0.010109521448612213, -0.026940487325191498, 0.022307438775897026, -0.022226588800549507, -0.1496925950050354, -0.017493003979325294, 0.007676261477172375, 0.054352227598428726, 0.003254449460655451, 0.031725913286209106, -0.0846213772892952, -0.029406029731035233, 0.05159557983279228, 0.0481240414083004, -0.0033148375805467367, -0.05827924236655235, 0.041969284415245056, 0.02221062034368515, 0.1281888782978058, -0.0223389845341444, -0.01165628433227539, 0.06292835623025894, -0.03287626802921295, -0.09122604876756668, -0.031175415962934494, 0.052699532359838486, 0.047034814953804016, -0.08420310914516449, -0.030056163668632507, -0.02074483595788479, 0.009517813101410866, -0.00372179

In [9]:
print(len(Embedding[0]))

384


In [10]:
import os
from pinecone import ServerlessSpec,Pinecone
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
index_name="scheduleai-index"
pc=Pinecone(api_key=PINECONE_API_KEY)
if index_name not in pc.list_indexes():
                    pc.create_index(
                    name=index_name,
                    dimension=384,
                    metric="cosine",
                    spec=ServerlessSpec(cloud="aws",region="us-east-1")
                )
index=pc.index(index_name)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-10', 'x-cloud-trace-context': 'e5f913060d62a390528dbe2fb42a97fe', 'date': 'Wed, 24 Dec 2025 02:07:06 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [12]:
from langchain_pinecone import PineconeVectorStore

vectorstore=PineconeVectorStore.from_documents(
    documents=splitted_doc,
    embedding=model,
    index_name=index_name
)